In [ ]:
# Beräknar antalet ratings per film
rating_per_movie = df_ratings.groupby('movieId').size().reset_index(name='num_ratings')
# Sorterar filmer efter nr ratings
sorted_movies = rating_per_movie.sort_values(by='num_ratings', ascending=False)
# 10 filmer med flest ratings
top_10_movies = sorted_movies.head(10)
print('Top 10 Movies with Most Ratings')
print(top_10_movies)

In [ ]:
def recommender(movie_name, model, n_recommendations):
    idx = process.extractOne(movie_name, df_movies["title"])[2]
    print("Movie Selected: ", df_movies["title"][idx], "Index: ", idx)
    print("Searching for recommendations....")
    distances, indices = model.kneighbors(sparse_matrix[idx].reshape(1, -1), n_neighbors=n_recommendations)
    for i in indices:
        print(df_movies["title"][i].where(i != idx))

recommender("fight club", model_knn,20)

In [ ]:
# Först, fyll NaN-värden med en tom sträng i kolumnen 'tag'
df_tags_filled = df_tags.fillna({'tag': ''})

# Filtrera filmer som tillhör en specifik genre
desired_genre = "Drama"  # Till exempel, vi väljer genren Drama
df_movies_genre = df_tags_filled[df_tags_filled['tag'].str.contains(desired_genre, case=False)]

# Skapa en ny DataFrame för betyg för filmer som tillhör den specifika genren
df_filtered_genre = df_ratings[df_ratings['movieId'].isin(df_movies_genre['movieId'])]

# Träna modellen och göra rekommendationer som vanligt
model_knn = NearestNeighbors(metric="cosine", algorithm="brute", n_neighbors=2)
model_knn.fit(df_filtered_genre)  # Träna modellen med data från den specifika genren

def recommender_genre(movie_name, model, n_recommendations):
    match = process.extractOne(movie_name, df_movies_genre["tag"])
    if match is not None:
        idx = match[2]
        print("Movie Selected: ", df_movies_genre["tag"].iloc[idx], "Index: ", idx)
        print("Searching for recommendations....")
        distances, indices = model.kneighbors(sparse_matrix[idx].reshape(1, -1), n_neighbors=n_recommendations)
        for i in indices:
            if i != idx and 0 <= i < len(df_movies_genre):  # Kontroll för att undvika ogiltiga index
                print(df_movies_genre["tag"].iloc[i])
            else:
                print("Invalid index:", i)
    else:
        print("Movie not found:", movie_name)

# Använd funktionen för att göra rekommendationer för en specifik film inom den valda genren
recommender_genre('Notebook', model_knn, 20)


In [ ]:
# Först, fyll NaN-värden med ''ND'' sträng i kolumnen 'tag'
df_tags_filled = df_tags.fillna({'tag': 'ND'})

# Skapa en ny DataFrame för betyg för filmer som tillhör den specifika genren
df_filtered_genre = df_ratings[df_ratings['movieId'].isin(df_movies['movieId'])]

def recommender_genre(movie_name, model, n_recommendations):
    match = process.extractOne(movie_name, df_tags_filled["tag"])
    if match is not None:
        idx = match[2]
        print("Movie Selected: ", df_tags_filled["tag"].iloc[idx], "Index: ", idx)
        print("Searching for recommendations....")
        distances, indices = model.kneighbors(sparse_matrix[idx].reshape(1, -1), n_neighbors=n_recommendations)
        for i in range(len(indices)):
            if indices[i] != idx and 0 <= indices[i] < len(df_tags_filled):  # Kontroll för att undvika ogiltiga index
                print(df_tags_filled["tag"].iloc[indices[i]])
            else:
                print("Invalid index:", indices[i])
    else:
        print("Movie not found:", movie_name)

# Använd funktionen för att göra rekommendationer för en specifik film inom den valda genren
recommender_genre('toy story', model_knn, 20)

In [ ]:
# Hitta liknande filmer med hjälp av KNN
def find_similar_movies(movie_title, X, k, metric='cosine', show_distance=False):
    neighbour_ids = []

# Hitta närmaste matchande filmtitel med hjälp av FuzzyWuzzy:

#     process.extractOne(movie_title, df_movies["title"]) söker efter den närmaste matchande filmtiteln för den användarvalda titeln med hjälp av FuzzyWuzzy-biblioteket.
#     Om en matchning hittas, returneras en tuple med matchande titel, matchpoäng och index för den matchande titeln.

    # Hämta film-id för den användarvalda titeln
    movie_match = process.extractOne(movie_title, df_movies["title"])

    # Hämta film-id för den matchande titeln:

    # Om en matchning hittas, extraheras den matchande titeln, matchpoängen och dess index från tupeln.
    # Filmens id hämtas från DataFrame df_movies med hjälp av indexet, och sedan lagras filmens id i variabeln selected_movie_id.

    if movie_match is not None:
        matched_title, score, idx = movie_match
        print(f"Closest match: {matched_title} (Score: {score})")
        selected_movie_id = df_movies.loc[idx, 'movieId']
        print(f"Selected movieId: {selected_movie_id}")

# Beräkna index för den matchande filmen i CSR-matrisen (movie_ind):

#     movie_ind = movie_mapper[selected_movie_id] används för att översätta filmens id till dess motsvarande index i CSR-matrisen.

        movie_ind = movie_mapper[selected_movie_id]
        movie_vec = X[movie_ind]

    #     Använd KNN för att hitta liknande filmer:

    # En instans av NearestNeighbors skapas med antalet grannar k och den valda metriken (metric).
    # CSR-matrisen X passas till fit()-metoden för att träna modellen.
    # Den matchande filmens vektor (movie_vec) i CSR-matrisen extraheras.
    # kNN.kneighbors(movie_vec, return_distance=show_distance) används för att hitta de närmaste k grannarna till den matchande filmen.
    # Om show_distance är satt till True, returneras även avståndet mellan den matchande filmen och dess grannar.

        k += 1
        kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
        kNN.fit(X)
        
        movie_vec = movie_vec.reshape(1, -1)
        neighbour = kNN.kneighbors(movie_vec, return_distance=show_distance)

    #     Översätt index för grannarna till film-id (neighbour_ids):

    # Indexen för grannarna returneras från kneighbors()-funktionen.
    # Dessa index översätts sedan till film-id med hjälp av mappningen movie_inv_mapper.
    # De översatta film-id:na lagras i listan neighbour_ids.

        for i in range(0, k):
            n = neighbour.item(i)
            neighbour_ids.append(movie_inv_mapper[n])
        neighbour_ids.pop(0)
    else:
        print("Movie not found.")

    #     Returnera listan med liknande filmers id (neighbour_ids):

    # Om en matchning hittades, returneras listan neighbour_ids som innehåller id för de liknande filmerna.
    # Om ingen matchning hittades, returneras en tom lista.

    return neighbour_ids


# Användarvald filmtitel
selected_movie_title = input("Vilken film vill du söka på?")

# Hitta liknande filmer baserat på användarens val
similar_movie_ids = find_similar_movies(selected_movie_title, X, k=10)
print(f"Since you watched {selected_movie_title}:")
for movie_id in similar_movie_ids:
    print(movie_titles[movie_id])
